In [1]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.dataset import *
from fastai.learner import *
from fastai.metrics import *
from fastai.core import *
from fastai.model import *
from fastai.lm_rnn import *
from collections import *
from fastai.conv_learner import *
import matplotlib.pyplot as plt

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

In [4]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from scipy.signal.windows import *

In [5]:
torch.cuda.set_device(0)

Kaggle Dog Breed Identification. Get data from https://www.kaggle.com/c/dog-breed-identification

In [6]:
PATH = "data/"
sz = 224
arch = resnext50
bs = 52

In [7]:
label_csv = f'{PATH}y3.csv'
n = len(list(open(label_csv))) - 1 # header is not counted (-1)
val_idxs = get_cv_idxs(n, val_pct=0.2) # random 20% data for validation set

In [8]:
#val_idxs = np.arange(int(n*.8), n)

In [9]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'vidya/person', f'{PATH}y3.csv', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

['class_0', 'class_1', 'class_10', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']
[2 2 2 ... 2 2 2]


In [10]:
size_d = {k: PIL.Image.open(PATH + k).size for k in data.trn_ds.fnames}

In [11]:
row_sz, col_sz = list(zip(*size_d.values()))

In [12]:
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

In [13]:
len(data.trn_ds)#, len(data.test_ds)

3040

In [14]:
len(data.classes), data.classes[:5]

(11, ['class_0', 'class_1', 'class_10', 'class_2', 'class_3'])

## Initial model

In [26]:
def get_data(sz, bs): # sz: image size, bs: batch size
    data = ImageClassifierData.from_csv(PATH, 'vidya/person', f'{PATH}y3.csv',
                                       val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/13
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/37
    return data if sz > 300 else data.resize(340, 'tmp') # Reading the jpgs and resizing is slow for big images, so resizing them all to 340 first saves time

#Source:   
#    def resize(self, targ, new_path):
#        new_ds = []
#        dls = [self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl]
#        if self.test_dl: dls += [self.test_dl, self.test_aug_dl]
#        else: dls += [None,None]
#        t = tqdm_notebook(dls)
#        for dl in t: new_ds.append(self.resized(dl, targ, new_path))
#        t.close()
#        return self.__class__(new_ds[0].path, new_ds, self.bs, self.num_workers, self.classes)
#File:      ~/fastai/courses/dl1/fastai/dataset.py

In [27]:
data.trn_ds.get_y(10)

2

### Precompute

In [28]:
data = get_data(sz, bs)

['class_0', 'class_1', 'class_10', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']
[2 2 2 ... 2 2 2]


In [33]:
learn = ConvLearner.pretrained(arch, data, precompute=False)

In [2]:
#%env CUDA_LAUNCH_BLOCKING=1

In [20]:
bs = 16

In [21]:
trn_dl = torch.utils.data.DataLoader(trn_ds, batch_size=bs)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=bs)

In [22]:
class ConvNet(nn.Module):
    def __init__(self, out_dim=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3))
        self.fc = nn.Linear(26*26*32, out_dim)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #print(out.size())
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [23]:
myConv = SingleModel(ConvNet().cuda())

In [24]:
md = ModelData('model', trn_dl, val_dl)

In [29]:
myConv = SingleModel(ConvnetBuilder(resnet18, 200, False, False).model)

In [30]:
myConv.model

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, ker

In [31]:
#myConv.model[:-1]

In [32]:
#learner = Learner(md, myConv, opt_fn=optim.Adam, crit=F.nll_loss)

In [33]:
#learner.fit(1*1e-3, 1)

In [34]:
class PasswordStatefulLSTM(nn.Module):
    def __init__(self, bs):
        super().__init__()
        
        self.cnn_final_width = 100
        self.number_rnn_layers = 1
        self.rnn_hidden_count = 20
        self.output_size = 11
        
        self.myConv = ConvNet(out_dim=11)
        
        self.convToLSTM = nn.Linear(115200, self.cnn_final_width)
        #self.convToLSTMBn = nn.BatchNorm1d(115200)
        
        self.flatten = Flatten()
        
        self.rnn = nn.LSTM(self.cnn_final_width, self.rnn_hidden_count, self.number_rnn_layers, dropout=0.2)
        self.l_out = nn.Linear(self.rnn_hidden_count, self.output_size)
        self.final_out = nn.Linear(self.cnn_final_width, self.output_size)
        self.init_hidden(bs)
        #self.conv = ConvNet(out_dim=self.cnn_final_width)
        self.conv = ConvnetBuilder(resnet18, 11, False, False).model#[:6]
        
    def forward(self, cs):
        #rnn_input = self.conv(cs).unsqueeze(1) #seq_len, batch, input_size
        #rnn_input = F.relu(self.convToLSTM(self.flatten(self.conv(cs))), inplace=True)
        #rnn_input = F.relu(rnn_input)
        #rnn_input = rnn_input.unsqueeze(1)
        
        #outp,h = self.rnn(rnn_input, self.h)
        #self.h = repackage_var(h)
        
        #------- wtf test
        #print(cs.shape)
        x = self.conv(cs)
        
        #print(x.shape)
        out = F.log_softmax(x, dim=-1)
        #print(out)
        return out
        
        
        #return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.output_size)
        #return F.log_softmax(self.final_out(rnn_input), dim=-1).view(-1, self.output_size)
        
    
    def init_hidden(self, bs):
        #self.h = (V(torch.zeros(self.number_rnn_layers, bs, self.rnn_hidden_count)),
        #          V(torch.zeros(self.number_rnn_layers, bs, self.rnn_hidden_count)))
        self.h = (V(torch.zeros(1, 1, self.rnn_hidden_count)),
                  V(torch.zeros(1, 1, self.rnn_hidden_count)))

In [35]:
myLSTM = SingleModel(PasswordStatefulLSTM(bs).cuda())

/home/robert/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [36]:
def faux_loss(preds, target):
    #print(preds, target)
    return F.nll_loss(preds, target)

In [42]:
learner = Learner(md, myLSTM, opt_fn=optim.SGD, crit=faux_loss)

In [43]:
#learner.lr_find()

In [44]:
#learner.sched.plot()

In [45]:
learner.fit(1e-2, 1, metrics=[accuracy])

epoch      trn_loss   val_loss   accuracy                     
    0      nan        nan        0.1       


[nan, 0.1]